In [ ]:
# файл криво называется, лучше такое на публику не показывать

In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import numpy as np

LEN = 1807  # хардкод, лучше избегать
            # если это количество акций - лучше придумать, как его доставать из страницы

In [2]:
URL = "https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=12"
# В URL видно, что есть полезные параметры, в частности start и end. Через них можно попытаться выгрузить например
# весь список акций, чтобы дальше не итерироваться по страницам

HEADERS = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36" }

In [328]:
def get_html(URL, params = None):  # PEP 8
    r = requests.get(URL, headers = HEADERS, params = params)
    return r

In [329]:
def parse(URL, _len):  # PEP 8
    """
    К функциям лучше прилагать описание
    """
    
    stock_params_dict = {}
    html = get_HTML(URL)
    
    if html.status_code == 200:
        soup = BeautifulSoup(html.text, "html.parser")
        items = soup.find_all("a", class_ = "Link-module__link__Wv1b Link-module__link_theme_default_gf5wq")
        
        # find_all можно оформить как find_all("a", {'class': '...'}), но так тоже норм
        
        # строки длинные, но это в целом не сильно критично
        
        stock_title_list = pd.Series([item.find('div', class_ = "Caption__caption_cyYZT") \
                         for item in items if item.find('div', class_ = "Caption__caption_cyYZT") ]).apply(lambda i: i.text)
        stock_ticker_list = pd.Series([item.find('div', class_ = "Caption__subcaption_xTAKS") \
                         for item in items if item.find('div', class_ = "Caption__subcaption_xTAKS") ]).apply(lambda i: i.text)
        stock_price_list = [price.replace("\xa0","") for price in pd.Series([item.find('div', class_ = "SecurityColumn__cellPriceSecurities_cswKs") \
                         for item in items if item.find('div', class_ = "SecurityColumn__cellPriceSecurities_cswKs") ]).apply(lambda i: i.text)]
        stock_yield_listok = pd.Series([item.find('span', class_ = "Money-module__money_UwC2N") \
                         for item in items if item.find('span', class_ = "Money-module__money_UwC2N") ]).apply(lambda i: i.text)  # не очень хороший нейминг
        stock_yield_list = [yld.replace("\xa0","") for yld in stock_yield_listok[::2]]  # ::2 - хардкод?
        
        for i in range(_len):
            stock_params_dict[stock_title_list[i]] = [stock_ticker_list[i],stock_price_list[i], stock_yield_list[i]]
            
    else:
        print("Error")
        
    return stock_params_dict

# иногда если if гораздо больше, чем else, их меняют местами (например заменить == 200 на != 200 и всё остальное
# убрать в else)

In [330]:
url_all = "https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=12"

# не очень хороший нейминг + PEP 8

In [332]:
stock_params_dict  = parse(URL_all, 12)

In [333]:
all_url = []  # PEP 8
for i in range(1,4):  # хардкод
    all_URL.append("https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=" + str(12*i))

In [334]:
df = np.transpose(pd.DataFrame(stock_params_dict))  
df.columns = ["Ticker", "Price", "Yield"]  # PEP 8
df["Price"] = df["Price"].map(lambda i: float(i[:-1].replace(",", "."))) # PEP 8

In [335]:
# [ parse(url) for url in all_URL]
# all_URL
data = parse("https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=" + str(LEN), LEN)

In [3]:
# Назначение кода сверху непонятно, очень не хватает комментов

In [362]:
def struct(data):
    """
    Описание
    """
    
    all_df = pd.DataFrame(data)
    all_df = np.transpose(all_df)
    all_df.columns = ["Ticker", "Price", "Nom_Yield"] # PEP 8
    all_df["Dir_Change"] = all_df["Nom_Yield"].apply(lambda i: i[0])
    all_df["Currency"] = all_df["Nom_Yield"].apply(lambda i: i[-1])
    all_df["Nom_Yield"] = all_df["Nom_Yield"].apply(lambda i: i[1:-1])
    all_df["Nom_Yield"] = all_df["Nom_Yield"].apply(lambda i: i.replace(",","."))
    
    # Разделение Nom_Yield наверняка можно оптимизировать, надо посмотреть на исходный датафрейм
    
    all_df["Price"] = all_df["Price"].apply(lambda i: i[:-1])
    
    return all_df

,Ticker,Price,Nom_Yield,Dir_Change,Currency
1-800-Flowers.com Inc,FLWS,"34,21",4.89,+,$
10X Genomics Inc,TXG,"160,27",8.24,−,$
1Life Healthcare,ONEM,"21,71",,0,$
2seventy bio,TSVT,"35,25",,0,$
2U Inc,TWOU,"25,83",10.07,−,$
...,...,...,...,...,...
Энел Россия,ENRU,"0,89",0.08,+,₽
ЭсЭфАй,SFIN,"454,6",44.6,−,₽
Юнипро,UPRO,"2,79",0.15,−,₽
ЮУНК,UNKL,8650,1320,−,₽


In [5]:
# Проверку кода можно сделать удобнее, если делать это через отдельную ветку и code review
# Перед внесением изменений делаешь git checkout -b dev (dev - любое название)
# Дальше там работаешь, коммитишь
# После этого git push --set-upstream origin mk_codereview_21_11_21
# Дальше в гитхабе ищешь branches, там ветку dev и запрашиваешь pull request
# Справа будет кнопка assignees - можешь назначить меня на проверку
# Дальше нажимаешь pull request и ждёшь